# Objetivo
Serviço que recebe lista de atuais clientes do usuário e retorna novos potenciais clientes. 

# Método para avaliação

1. Clusterizar banco completo usando **Agglomerative Clustering + KNN**. Com Agglomerative, geram-se os labels para 2% do banco (mais do que isso, dá erro de memória). Depois, KNN entra como classificador supervisionado para clusterizar os restantes 98% do banco. O resultado é uma classificação: a qual cluster cada cliente pertence
2. Ler arquivo com lista de IDs de clientes atuais
3. Separar parte para teste
4. Identificar clusters da lista de clientes atuais
5. Buscar novos clientes dos mesmos clusters no banco completo
6. Avaliar se lista de recomendações/sugestões inclui a parte separada para teste

Versão final deve ser:
1. Ler arquivo com lista de IDs de clientes atuais
2. Ler banco clusterizado
3. Identificar clusters da lista de clientes atuais
4. Buscar novos clientes dos mesmos clusters no banco completo
5. Apresentar resultados

# Imports

In [1]:
# imports
import pandas as pd
import numpy as np
import feather

# Carregamento do banco de dados com colunas filtradas

In [2]:
fullmarket = feather.read_dataframe('full_market_reduced_columns.feather')
fullmarket.set_index('id',inplace=True)
fullmarket.info()
#Tamanho já mais razoável: 141 MB. Ainda tem que resolver os NaN

<class 'pandas.core.frame.DataFrame'>
Index: 462298 entries, a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849 to 3d43e934e150b86be1e67524f5ba1018b27da9ef25566d9c0607623ae7f25e3a
Data columns (total 47 columns):
fl_matriz                                462298 non-null bool
de_natureza_juridica                     462298 non-null object
sg_uf                                    462298 non-null object
natureza_juridica_macro                  462298 non-null object
de_ramo                                  462298 non-null object
setor                                    460371 non-null object
idade_empresa_anos                       462298 non-null float64
idade_emp_cat                            462298 non-null object
fl_me                                    462298 non-null bool
fl_sa                                    462298 non-null bool
fl_epp                                   462298 non-null bool
fl_mei                                   462298 non-null bool
fl_ltda       

In [3]:
fullmarket.head(3)

,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,...,qt_socios,qt_socios_pj,idade_media_socios,qt_socios_masculino,qt_socios_feminino,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais
id,,,,,,,,,,,,,,,,,,,,,
a6984c3ae395090e3bee8ad63c3758b110de096d5d819583a784a113726db849,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,...,2.0,0.0,44.0,2.0,NaN,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,3132172.8,0
6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c47fdef50b33f4f56f05,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,False,...,1.0,0.0,27.0,1.0,NaN,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0
4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02edc5147bc2a2535b08,True,EMPRESARIO INDIVIDUAL,AM,OUTROS,"TRANSPORTE, ARMAZENAGEM E CORREIO",SERVIÇO,7.093151,5 a 10,False,False,...,1.0,0.0,32.0,1.0,NaN,"ATE R$ 81.000,00","ATE R$ 81.000,00",50000.0,50000.0,0


# Seleção de features e clusterização

In [4]:
feature_list = ['de_nivel_atividade', 'de_ramo', 'nm_meso_regiao', 'setor']

In [5]:
# fillna
for feat in feature_list:
    print(feat, '-->>', len(fullmarket[feat].unique().tolist()), ' categorias')
    print(fullmarket[feat].value_counts())
    print('NaN antes: ', sum(fullmarket[feat].isna()))
    fullmarket[feat].fillna('DESCONHECIDO', inplace = True)
    print('NaN depois: ', sum(fullmarket[feat].isna()))
    print('\n')

de_nivel_atividade -->> 5  categorias
MEDIA          217949
ALTA           152245
BAIXA           76080
MUITO BAIXA      4856
Name: de_nivel_atividade, dtype: int64
NaN antes:  11168
NaN depois:  0


de_ramo -->> 33  categorias
COMERCIO VAREJISTA                                   172404
SERVICOS DIVERSOS                                     60318
SERVICOS DE ALOJAMENTO/ALIMENTACAO                    30180
INDUSTRIA DA CONSTRUCAO                               25363
COMERCIO E REPARACAO DE VEICULOS                      22356
SERVICOS ADMINISTRATIVOS                              21326
BENS DE CONSUMO                                       21227
SERVICOS PROFISSIONAIS, TECNICOS E CIENTIFICOS        17371
COMERCIO POR ATACADO                                  16464
TRANSPORTE, ARMAZENAGEM E CORREIO                     15485
SERVICOS DE EDUCACAO                                  11311
SERVICOS DE SAUDE                                      9788
CULTURA, ESPORTE E RECREACAO                        

## Encoder

In [6]:
# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [7]:
# apply le on categorical feature columns
# https://towardsdatascience.com/encoding-categorical-features-21a2651a065c

fullmarket_enc=pd.DataFrame()
categorical_cols = ['de_nivel_atividade', 'de_ramo', 'nm_meso_regiao', 'setor']

fullmarket_enc[categorical_cols] = fullmarket[categorical_cols].apply(lambda col: le.fit_transform(col))

## Clusterização

Problema com AgglomerativeClustering: não há memória suficiente:
https://stackoverflow.com/questions/51129498/memory-error-during-hierarchical-clustering-python-3-6

https://datascience.stackexchange.com/questions/47889/how-to-run-agglomerativeclustering-on-a-big-data-in-python

Sugestão é amostrar o banco e usar KNN.

Ou usar outro algoritmo, como MiniBatchKMeans ou Birch.
https://github.com/scikit-learn/scikit-learn/issues/5256

https://chrisalbon.com/machine_learning/clustering/minibatch_k-means_clustering/

https://towardsdatascience.com/machine-learning-birch-clustering-algorithm-clearly-explained-fb9838cbeed9

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html

Birch clustering algorithm is a memory-efficient, online-learning algorithm provided as an alternative to MiniBatchKMeans.

**Solução utilizada**: Agglomerative Clustering em 2% do banco seguido por KNN.


In [8]:
#Então, vamos amostrar o banco
from sklearn.model_selection import train_test_split
#Usamos só o X. y é descartado, porque não há variável-alvo
train_size_agglom = 0.02 # 2% do market aprox. 10000 amostras
sample_market, market_minus_sample, y_train, y_test = train_test_split(fullmarket_enc,
                                                        pd.Series(np.zeros(len(fullmarket_enc))),
                                                        train_size=train_size_agglom,
                                                        random_state=17) 

from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=500, affinity='euclidean', linkage='ward')
sample_market_cluster_label = cluster.fit_predict(sample_market)
# Com 10000, ok

Precisamos do banco completo clusterizado. Como o AgglomerativeClustering não funciona por limitação de memória, vamoa adotar a estratégia de clusterizar uma parte e depois usar KNN para predizer as classes do resto do banco.

In [9]:
from sklearn.neighbors import KNeighborsClassifier
KN = KNeighborsClassifier(n_neighbors=3)
KN.fit(sample_market, sample_market_cluster_label)
clusters_market_minus_sample = KN.predict(market_minus_sample)

In [10]:
market_minus_sample_with_clusters = market_minus_sample.assign(cluster = clusters_market_minus_sample)
sample_market_with_clusters = sample_market.assign(cluster = sample_market_cluster_label)
fullmarket_with_clusters = pd.concat([sample_market_with_clusters, market_minus_sample_with_clusters])

Resultado: banco completo, com informação (label) de cluster, em `fullmarket_with_clusters`

# Carregamento lista de clientes atuais
Formato: arquivo csv, com IDs dos clientes

In [11]:
current_customers = pd.read_csv('estaticos_portfolio3.csv')
if not 'id' in current_customers.columns:
    print('Error. Check yout file. It should contain a column named "id"') 
else:
    current_customers.set_index('id',inplace=True)
    print('Ok. Loaded file. Shape:', current_customers.shape)    

Ok. Loaded file. Shape: (265, 1)


# Avaliação

In [12]:
# Verificar número de hits (quantas sugestões são parte de um conjunto de teste)
def count_hits_in_suggestion(df_suggestions, df_X_test): #suggestions e X_test são dataframes
    return sum(df_X_test.index.isin(df_suggestions.index))

In [13]:
# Para verificar se os clientes estão no banco
count_hits_in_suggestion(current_customers, fullmarket_with_clusters)

265

## Separação "treinamento" e teste

Esta separação deve ser usada somente para avaliação, não para a versão final.

In [15]:
from sklearn.model_selection import train_test_split
# ignorar y. Só usamos X
X_train, X_test, y_train, y_test = train_test_split(
    current_customers, list(range(len(current_customers))), test_size=0.3, random_state=17)
print(f'Tamanho do "treinamento": {X_train.shape}')
print(f'Tamanho do teste para avaliação: {X_test.shape}')

Tamanho do "treinamento": (185, 1)
Tamanho do teste para avaliação: (80, 1)


In [16]:
curr_customers_in_base = fullmarket_with_clusters[fullmarket_with_clusters.index.isin(X_train.index)]
curr_customers_in_base.shape

(185, 5)

In [17]:
clusters_ranking = curr_customers_in_base['cluster'].value_counts()

In [28]:
most_frequent_cluster = clusters_ranking.index[0]

In [29]:
#sugestões:
suggestions = fullmarket_with_clusters[fullmarket_with_clusters['cluster']==most_frequent_cluster]
len(suggestions)

245

In [30]:
our_hits = count_hits_in_suggestion(suggestions, X_test)
print(our_hits)

8


In [31]:
# Compara com sorteio aleatório (random picking) da base market

n_picking = len(suggestions) # quantidade de sugestões a sortear

# Simulação de Monte Carlo, ou seja, sortear um conjunto várias vezes e depois fazer a média
monte_carlo_size = 500
num_hits = int(0)
for k in range(monte_carlo_size):
    random_suggestion = fullmarket_with_clusters.sample(n = n_picking)
    num_hits = num_hits + count_hits_in_suggestion(random_suggestion, X_test)
mean_num_hits = num_hits / monte_carlo_size

In [32]:
print(f'Number of hits in our suggestion: {our_hits}.')
print(f'They represent {100*our_hits/len(X_test):.2f}% of the test size.\n')

print(f'For random suggestion, we found an average of {mean_num_hits:.2f} hits.')
print(f'They represent {100*mean_num_hits/len(X_test):.2f}% of the test size.')

Number of hits in our suggestion: 8.
They represent 10.00% of the test size.

For random suggestion, we found an average of 0.03 hits.
They represent 0.04% of the test size.
